### Librerías

In [2]:
import pandas as pd  # For data manipulation and analysis
import random  # For generating random numbers
import itertools  # For efficient looping
import os  # For interacting with the operating system
import numpy as np  # For numerical operations
import scipy.stats as stats  # For statistical functions
import seaborn as sns  # For statistical data visualization
import matplotlib.pyplot as plt  # For creating static, interactive, and animated visualizations
from scipy.stats import ttest_ind, norm  # For conducting t-tests and working with normal distributions
from sklearn.metrics import mean_absolute_error, mean_squared_error  # For evaluating model performance
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.ensemble import RandomForestRegressor  # For using Random Forest regression models
from sklearn.linear_model import Lasso  # For using Lasso regression models
from sklearn.preprocessing import StandardScaler  # For feature scaling

import statsmodels.api as sm  # For statistical models and tests
import statsmodels.formula.api as smf  # For statistical models with formula interface
from linearmodels.panel import PanelOLS  # For panel data models
from tqdm import tqdm  # For adding progress bars to loops

# Set the random seed for reproducibility
random.seed(123)

# Initialize the scaler
scaler = StandardScaler()


def formula_add(data):
    text = ""
    for i in data:
        text = f"{text} + {i}"
    
    return text

### Ugo database processing

In [3]:
ugo = pd.read_csv("INPUT/WEO_1990_2022_FR.csv")

In [4]:
c_inflacion = ["cpi_1990_a_a", "cpi_1990_a_f0", "cpi_1990_a_f1", "cpi_1990_a_f2", "cpi_1990_a_f3"]
c_ur = ["ur_a", "ur_f0", "ur_f1", "ur_f2", "ur_f3"]
c_ir = ["ir_a", "ir_f0", "ir_f1", "ir_f2", "ir_f3"]
c_exr = ["exr_a", "exr_f0", "exr_f1", "exr_f2", "exr_f3"]
c_pusd = ["gdp_usd_a", "gdp_usd_f0", "gdp_usd_f1", "gdp_usd_f2", "gdp_usd_f3"]

inf = ugo[["country", "ifscode", "year"] + c_inflacion + c_pusd]

inf["cpi_porcentaje"] = inf.groupby('ifscode')['cpi_1990_a_a'].pct_change()
inf = inf[np.abs(inf["cpi_porcentaje"])<1]

inf = inf.dropna()

# #DESVIACIONES DEL VALOR OBSERVADO PARA INFLACIÓN
c_dinflacion = []
tolerancia = 500

for columna in c_inflacion[0:]:
    # percentil95 = inf[columna].quantile(0.85)
    # print(percentil95)
    inf = inf[inf[columna]<tolerancia]
    
    nombre = f"desv_{columna}"
    c_dinflacion.append(nombre)

    inf[nombre] = (inf["cpi_1990_a_a"]/inf[columna])-1

inf = inf.rename(columns = {"ifscode": "WEO_CODE", "year": "DATE"})
inf.columns = [i.upper() for i in inf.columns]
# X_INF = ["DESV_CPI_1990_A_F1","DESV_CPI_1990_A_F2", "DESV_CPI_1990_A_F3"]
X_INF = ["DESV_CPI_1990_A_F0", "DESV_CPI_1990_A_F1", "DESV_CPI_1990_A_F2", "DESV_CPI_1990_A_F3"] # -> CONFIGURACIÓN FINAL
# X_INF= ["DESV_CPI_1990_A_F3"]

inf_pn = inf.dropna()

<ipython-input-4-840fe013a060>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inf["cpi_porcentaje"] = inf.groupby('ifscode')['cpi_1990_a_a'].pct_change()


In [5]:
inf_pn

,COUNTRY,WEO_CODE,DATE,CPI_1990_A_A,CPI_1990_A_F0,CPI_1990_A_F1,CPI_1990_A_F2,CPI_1990_A_F3,GDP_USD_A,GDP_USD_F0,GDP_USD_F1,GDP_USD_F2,GDP_USD_F3,CPI_PORCENTAJE,DESV_CPI_1990_A_A,DESV_CPI_1990_A_F0,DESV_CPI_1990_A_F1,DESV_CPI_1990_A_F2,DESV_CPI_1990_A_F3
6,United States,111.0,1996.0,120.050980,119.939194,120.584419,121.336617,120.567924,7661.575195,7582.589844,7411.670898,7481.014648,7431.866699,0.029458,0.0,0.000932,-0.004424,-0.010596,-0.004288
7,United States,111.0,1997.0,119.585770,122.952484,123.349358,124.676842,125.462059,8110.925000,8091.333984,7941.623535,7813.339355,7889.366211,-0.003875,0.0,-0.027382,-0.030512,-0.040834,-0.046837
8,United States,111.0,1998.0,121.229508,124.867264,126.341293,127.058998,128.915863,8790.175000,8499.491211,8514.143555,8317.386719,8238.889648,0.013745,0.0,-0.029133,-0.040460,-0.045880,-0.059623
9,United States,111.0,1999.0,123.546448,123.550512,127.715210,130.131531,130.870758,9268.600000,8956.490236,8843.205078,8937.152344,8712.878906,0.019112,0.0,-0.000033,-0.032641,-0.050603,-0.055966
10,United States,111.0,2000.0,127.255824,127.064647,126.230096,130.812714,134.035477,9824.650000,9996.244501,9400.428618,9238.809570,9380.381836,0.030024,0.0,0.001505,0.008126,-0.027191,-0.050581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6080,Romania,968.0,2018.0,219.224813,219.261184,217.437241,217.932585,219.989615,241.625932,239.440441,233.485817,210.100094,211.093950,0.024669,0.0,-0.000166,0.008221,0.005929,-0.003477
6081,Romania,968.0,2019.0,124.222096,124.222096,128.286978,131.997095,135.362111,243.697950,243.697950,261.867542,281.061894,301.666223,-0.433358,0.0,0.000000,-0.031686,-0.058903,-0.082298
6082,Romania,968.0,2020.0,127.395664,127.395664,130.584470,134.073921,137.656617,248.623788,248.623788,289.407724,312.942048,336.942182,0.025548,0.0,0.000000,-0.024419,-0.049810,-0.074540
6083,Romania,968.0,2021.0,132.582626,132.582626,137.070565,141.117885,144.794493,287.279238,287.279238,314.876337,345.318714,372.625277,0.040715,0.0,0.000000,-0.032742,-0.060483,-0.084339


### Model estimations

In [6]:
X = inf_pn[X_INF]
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns).reset_index(drop = True)
y = inf_pn['CPI_PORCENTAJE'].reset_index(drop = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)


In [9]:
X.sort_values("DESV_CPI_1990_A_F0")

,DESV_CPI_1990_A_F0,DESV_CPI_1990_A_F1,DESV_CPI_1990_A_F2,DESV_CPI_1990_A_F3
3294,-8.244118,-6.426683,-5.642955,-4.964505
3293,-8.231058,-6.764416,-5.629985,-4.956671
1891,-7.076699,-5.468502,-4.531883,-3.969437
1892,-6.445676,-4.967961,-4.109102,-3.592599
752,-5.719615,-4.369786,-3.692027,-3.197789
...,...,...,...,...
1893,5.340815,-4.522375,-3.732740,-3.257134
3145,6.697683,5.918387,5.665487,5.090964
3752,8.329247,7.646074,6.403098,5.948045
3753,8.597954,7.051720,6.874454,6.065858


In [10]:
X = inf_pn[X_INF]

In [12]:
X.sort_values("DESV_CPI_1990_A_F0")

,DESV_CPI_1990_A_F0,DESV_CPI_1990_A_F1,DESV_CPI_1990_A_F2,DESV_CPI_1990_A_F3
4346,-0.891081,-0.895311,-0.942304,-0.943131
4345,-0.889718,-0.939737,-0.940284,-0.941762
2471,-0.769270,-0.769270,-0.769270,-0.769270
2472,-0.703428,-0.703428,-0.703428,-0.703428
884,-0.627670,-0.624743,-0.638475,-0.634446
...,...,...,...,...
2473,0.526395,-0.644815,-0.644815,-0.644815
4142,0.667973,0.728579,0.818824,0.813783
5012,0.838213,0.955842,0.933696,0.963534
5013,0.866250,0.877660,1.007103,0.984119


In [4]:
################################################################### ML MODELS
# X = inf[X_INF + C_INS + C_DEUDA_L]
X = inf_pn[X_INF]
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns).reset_index(drop = True)
y = inf_pn['CPI_PORCENTAJE'].reset_index(drop = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

########### LASSO IV
alphas = np.arange(0, 1, 0.001)

best_alpha = None
best_mse = np.inf
best_model = None

for alpha in alphas:
    lasso_model = sm.OLS(y_train, X_train).fit_regularized(alpha=alpha, L1_wt=1.0)
    y_pred_lasso = lasso_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_lasso)
    
    if mse < best_mse:
        best_mse = mse
        best_alpha = alpha
        best_model = lasso_model

# Print the best alpha and evaluate the best model
print("Best Alpha:", best_alpha)
model = sm.OLS(y*100, X).fit_regularized(alpha=best_alpha, L1_wt=1)
y_lasso = model.predict(X)/100

########### RIDGE IV
alphas = np.arange(0, 1, 0.001)

best_alpha = None
best_mse = np.inf
best_model = None

for alpha in alphas:
    lasso_model = sm.OLS(y_train, X_train).fit_regularized(alpha=alpha, L1_wt=0)
    y_pred_lasso = lasso_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_lasso)
    
    if mse < best_mse:
        best_mse = mse
        best_alpha = alpha
        best_model = lasso_model

# Print the best alpha and evaluate the best model
print("Best Alpha:", best_alpha)
model = sm.OLS(y*100, X).fit_regularized(alpha=best_alpha, L1_wt=0)
y_ridge = model.predict(X)*100


########### RANDOM FOREST IV
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_rf = model.predict(X)


########### PANEL OLS IV
# inf_pn = dfg[list(inf.columns) + C_INS + C_DEUDA_L].dropna()
inf_panel = inf_pn.set_index(['WEO_CODE', 'DATE'])
# formula = f"CPI_PORCENTAJE ~  {formula_add(X_INF)}  {formula_add(C_DEUDA_L)}  {formula_add(C_INS)} + EntityEffects + TimeEffects"
# formula = f"CPI_PORCENTAJE ~  {formula_add(X_INF)}"
# formula = f"CPI_PORCENTAJE ~  DESV_CPI_1990_A_F1 + DESV_CPI_1990_A_F2 + DESV_CPI_1990_A_F3 + EntityEffects + TimeEffects"
formula = f"CPI_PORCENTAJE ~  {formula_add(X_INF)} + EntityEffects + TimeEffects"
modelols = PanelOLS.from_formula(formula, data=inf_panel )
results = modelols.fit()
y_ols = model.predict(X)

Best Alpha: 0.002
Best Alpha: 0.20400000000000001


In [11]:
a = results.summary

In [16]:
a.as_html

<bound method Summary.as_html of <class 'linearmodels.compat.statsmodels.Summary'>
"""
                          PanelOLS Estimation Summary                           
Dep. Variable:         CPI_PORCENTAJE   R-squared:                        0.2705
Estimator:                   PanelOLS   R-squared (Between):             -0.1485
No. Observations:                4297   R-squared (Within):               0.1965
Date:                Tue, Apr 02 2024   R-squared (Overall):              0.1119
Time:                        15:05:11   Log-likelihood                    4813.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      377.48
Entities:                         192   P-value                           0.0000
Avg Obs:                       22.380   Distribution:                  F(4,4072)
Min Obs:                       2.0000                                           
Max Obs:              

In [14]:
#################################### DICTIONARY WITH FINAL DATA
actual_values = inf_panel['CPI_PORCENTAJE']

data = {
    'pi_por': actual_values.values,
    'pi_rf': y_rf,
    'pi_ols': y_ols,
    'pi_ridge': y_ridge,
    'pi_lasso': y_lasso
    
}

# Create a DataFrame
inflacion = pd.DataFrame(data)
inflacion.index = inf_panel.index
inflacion = inflacion.reset_index()
inf2 = inf[['WEO_CODE', 'DATE']+X_INF]
inflacion.rename(columns = {"ifscode": "WEO_CODE", "year": "DATE"}, inplace = True)
inflacion.columns = [i.upper() for i in inflacion.columns]
inflacion = inflacion.merge(inf2, "left", ['WEO_CODE', 'DATE'])

In [15]:
inflacion[["WEO_CODE", "DATE", "PI_POR", "PI_OLS"]].to_excel("INPUT/INFLATION_ACTUAL_PREDICTED_IV.xlsx", index = False)

<ipython-input-15-19eebd7a04d0>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  inflacion[["WEO_CODE", "DATE", "PI_POR", "PI_OLS"]].to_excel("INPUT/INFLATION_ACTUAL_PREDICTED_IV.xlsx", index = False)
